# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: Credit risk modelling using Logistic Regression

## Problem Statement

`Predict the loan defaulters` using a Logistic Regression model on the `credit risk data` and `calculate credit scores`

## Learning Objectives

At the end of the mini-project, you will be able to :

* perform data exploration, preprocessing and visualization
* implement Logistic Regression using manual code or using sklearn library
* evaluate the model using appropriate performance metrics
* develop a credit scoring system

## Dataset

The dataset chosen for this mini-project is the [Give Me Some Credit](https://cdn.iisc.talentsprint.com/CDS/Give_me_some_credit_BigML.pdf) dataset which can be used to build models for predicting loan repayment defaulters
#### Datafields

- **SeriousDlqin2yrs:** Person experienced 90 days past due delinquency or worse
- **RevolvingUtilizationOfUnsecuredLines:** Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits
- **age:** Age of borrower in years
- **NumberOfTime30-59DaysPastDueNotWorse:** Number of times borrower has been 30-59 days past due but no worse in the last 2 years.
- **DebtRatio:** Monthly debt payments, alimony,living costs divided by monthy gross income
- **MonthlyIncome:** Monthly income
- **NumberOfOpenCreditLinesAndLoans:** Number of Open loans (installment like car loan or mortgage) and Lines of credit (e.g. credit cards)
- **NumberOfTimes90DaysLate:** Number of times borrower has been 90 days or more past due.
- **NumberRealEstateLoansOrLines:**	Number of mortgage and real estate loans including home equity lines of credit
- **NumberOfTime60-89DaysPastDueNotWorse:**	Number of times borrower has been 60-89 days past due but no worse in the last 2 years.
- **NumberOfDependents:** Number of dependents in family excluding themselves (spouse, children etc.)

## Information

Credit risk arises when a corporate or individual borrower fails to meet their debt obligations. From the lender's perspective, credit risk could disrupt its cash flows or increase collection costs, since the lender may be forced to hire a debt collection agency to enforce the collection. The loss may be partial or complete, where the lender incurs a loss of part of the loan or the entire loan extended to the borrower.

Credit scoring algorithms, which calculate the probability of default, are the best methods that banks use to determine whether or not a loan should be granted.

In order to build a credit scoring system, the following feature transformations are performed:

#### Weight of Evidence and Information value

Logistic regression is a commonly used technique in credit scoring for solving binary classification problems. Prior to model fitting, another iteration of variable selection is valuable to check if the newly WOE transformed variables are still good model candidates. Preferred candidate variables are those with higher information value having a linear relationship with the dependent variable, have good coverage across all categories, have a normal distribution, contain a notable overall contribution, and are relevant to the business.

**Weight of evidence** (WOE) is a powerful tool for feature representation and evaluation in data science. WOE can provide interpret able transformation to both categorical and numerical features. The weight of evidence tells the predictive power of an independent variable in relation to the dependent variable. Since it evolved from credit scoring world, it is generally described as a measure of the separation of good and bad customers. "Bad Customers" refers to the customers who defaulted on a loan. and "Good Customers" refers to the customers who paid back loan. WOE can be calculated using the below formula:

$$WOE = ln \left( \frac{\%   of  Non\_Events}{\%   of  Events} \right)$$

Steps to calculate WOE
* For a continuous variable, split data into 10 parts (or lesser depending on the distribution).
* Calculate the number of events and non-events in each group (bin)
* Calculate the % of events and % of non-events in each group.
* Calculate WOE by taking natural log of division of % of non-events and % of events

**Information value** is one of the most useful technique to select important variables in a predictive model. It helps to rank variables on the basis of their importance. The IV is calculated using the following formula :
$$IV = ∑ (\% of Non\_Events - \% of Events) * WOE$$

Read more about `WOE` and `IV` from the following [link](https://cdn.iisc.talentsprint.com/CDS/Assignments/Module2/M2_NB_MiniProject_2_Credit_risk_modelling_Logistic_Regression_WoE_and_IV.pdf)

## Grading = 10 Points

### Download the dataset

In [ ]:
# @title Download Dataset
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/GiveMeSomeCredit.csv
#!pip -qq install xverse
print("Data Downloaded Successfully!")


### Install Packages

In [50]:
#!pip install pandas==1.3.5


In [51]:
#!pip install xverse


### Import Neccesary Packages

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns
from matplotlib import pyplot as plt
import math
from xverse.transformer import MonotonicBinning, WOE
from sklearn.preprocessing import StandardScaler


%matplotlib inline


### Load the dataset

In [ ]:
# YOUR CODE HERE
df = pd.read_csv("GiveMeSomeCredit.csv")
df.head()


#### Describe the all statistical properties of the train dataset

In [ ]:
# YOUR CODE HERE
df.describe()


In [ ]:
df.info()


### Pre-processing (1 point)

#### Remove unwanted columns

In [56]:
# YOUR CODE HERE
df.drop("Unnamed: 0", axis=1, inplace=True)


#### Handle the missing data

Find the how many null values in the dataset and fill with mean or remove.

In [ ]:
# YOUR CODE HERE
df.isnull().sum()


In [58]:
df["MonthlyIncome"] = df["MonthlyIncome"].fillna(df["MonthlyIncome"].median())
df["NumberOfDependents"] = df["NumberOfDependents"].fillna(
    df["NumberOfDependents"].median()
)


In [ ]:
df.isnull().sum()


### EDA &  Visualization ( 1 point)

#### Assumption
1. feature `SeriousDlqin2yrs` is a categorical variable and has two values with the meaning:
    1. `0` : person is a not a defaulter
    1. `1` : person is a defaulter
1. All features except `SeriousDlqin2yrs` are numerical variable


#### Calculate the percentage of the target labels and visualize with a graph

In [60]:
sns.set_theme(style="whitegrid", palette="pastel")


In [ ]:
# YOUR CODE HERE
target_value_count = df["SeriousDlqin2yrs"].value_counts()
print(target_value_count)

# target_value_count.plot(kind="bar")
non_defaulter, defaulter = target_value_count[0], target_value_count[1]
y = np.array([non_defaulter, defaulter])
y_percent = np.round(y / y.sum() * 100, 2)
x = np.array(["Non-Defaulter", "Defaulter"])

plt.figure(figsize=(8, 6))
plt.bar(x, y)
plt.text(
    0,
    non_defaulter,
    non_defaulter,
    # f"{(non_defaulter / (non_defaulter + defaulter)) * 100:.3f}%",
    fontdict={
        "fontsize": 12,
        "fontweight": "bold",
        "color": "black",
        "ha": "center",
        "va": "bottom",
    },
)
plt.text(
    1,
    defaulter,
    defaulter,
    # f"{(defaulter / (non_defaulter + defaulter)) * 100:.3f}%",
    fontdict={
        "fontsize": 12,
        "fontweight": "bold",
        "color": "black",
        "ha": "center",
        "va": "bottom",
    },
)
plt.xlabel("Credit Status")
plt.ylabel("Number of People")
plt.title("Credit Defaulter Status")
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
plt.pie(y, labels=x, autopct="%1.1f%%", explode=[0.2, 0], shadow=True, startangle=0)
plt.legend(title="Credit Status", loc="upper right")
plt.show()


#### Plot the distribution of SeriousDlqin2yrs by age

In [ ]:
unique_ages = df["age"].nunique()
print(f"Unique ages: {unique_ages}")


In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(
    data=df,
    x="age",
    hue="SeriousDlqin2yrs",
    multiple="stack",
    bins=unique_ages,
    kde=True,
)
plt.title("Distribution of SeriousDlqin2yrs by Age")
plt.xlabel("Age")
plt.ylabel("Count")
plt.legend(title="SeriousDlqin2yrs", labels=["Defaulter", "Non-Defaulter"])
plt.show()


#### Calculate the correlation and plot the heatmap

In [ ]:
# YOUR CODE HERE

# keep a copy of the original dataframe
df_ = df.copy(deep=True)
df_.head()


In [ ]:
# segregate the continuous and categorical variables
all_columns = df.columns.tolist()
print(f"All Columns: {all_columns}")

df_cat_var_columns = ["SeriousDlqin2yrs"]
df_cont_var_columns = [val for val in all_columns if val not in df_cat_var_columns]

print(f"Categorical Variables: {df_cat_var_columns}")
print(f"Continuous Variables: {df_cont_var_columns}")


In [ ]:
# apply the standard scaler

ss = StandardScaler()
df[df_cont_var_columns] = ss.fit_transform(df[df_cont_var_columns])
df.head()


In [ ]:
# plot the heat map
sns.set_theme(style="whitegrid", palette="pastel")
plt.figure(figsize=(12, 6))
sns.heatmap(df.corr(), annot=True, linewidth=0.5, center=0, cmap="coolwarm")
plt.show()


#### Observations
1. `age` has the lowest corelation with defaulting on credit.
1. Historically, people who have cleared there installments are more likely to default again:
    1. 30 to 59 days post due date
    1. 60 to 89 days after due date 
    1. exactly 90 days after due date
1. `NumberOfTime30-59DaysPastDueNotWorse`, `NumberOfTime60-89DaysPastDueNotWorse` and `NumberOfTimes90DaysLate` are highly correlated and hence only one of the field is sufficient to carry out the prediction.

##### Questions
1. Corelation between age and defaulter was counter intuitive for me. Can you confirm if the observation is correct?
1. Can we draw some parallels between the SHAPley Heatmap and Corelation Heatmap?

<hr/>

### Data Engineering (2 points)

#### Weight of Evidence and Information value

* Arrange the binning for each variable with different bins
    * For eg. Age = 49, Age_quantile_range = (48, 56)
* Calculate information value and chooose the best features based on the rules given below

| Information Value |	Variable Predictiveness |
| --- | --- |
| Less than 0.02	|  Not useful for prediction |
| 0.02 to 0.1	| Weak predictive Power |
|  0.1 to 0.3 | Medium predictive Power |
| 0.3 to 0.5 | Strong predictive Power |
| >0.5 | Suspicious Predictive Power |

* Calculate Weight of evidence for the selected variables

Hint: Use [xverse](https://cdn.iisc.talentsprint.com/CDS/Assignments/Module2/M2_NB_MiniProject_2_Credit_risk_modelling_Logistic_Regression_Xverse.pdf). It is a machine learning Python module in the space of feature engineering, feature transformation and feature selection. It provides pre-built functions for the above steps, such as binning and conversion to WoE.

In [ ]:
# YOUR CODE HERE
# separate the target variable and the features
y = df["SeriousDlqin2yrs"]
X = df.drop("SeriousDlqin2yrs", axis=1)

print(f"features : {X.columns}")
print(f"target : {y.name}")


In [ ]:
clf = MonotonicBinning()

# It's a 6 years old code. Is it maintained? https://github.com/Sundar0989/XuniVerse/tree/master
# My code doesn't compile with the current version of the package. I am commenting this code.

# clf.fit(X, y)
# print(clf.bins)  # print the bins

# output_bins = clf.bins  # store the bins in a variable
# clf = MonotonicBinning(
#     custom_binning=output_bins
# )  # initialize the MonotonicBinning class

# out_X = clf.transform(X)  # transform the features
# out_X.head()


In [ ]:
clf = WOE()
# clf.fit(X, y)

# clf.woe_df  # print the WOE values
# clf.iv_df  # print the IV values


### Identify features,  target and split it into train and test

In [89]:
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=101, stratify=y
)


### Logistic Regression from scratch using gradient method (2 points)

For Linear Regression, we had the hypothesis $yhat = w.X +b$ , whose output range was the set of all Real Numbers.
Now, for Logistic Regression our hypothesis is  $yhat = sigmoid(w.X + b)$ , whose output range is between 0 and 1 because by applying a sigmoid function, we always output a number between 0 and 1.

$yhat = \frac{1}{1 +e^{-(w.x+b)}}$

Hint: [logistic-regression-with-python](https://cdn.iisc.talentsprint.com/CDS/Assignments/Module2/odsc%20%20Logistic.pdf)

In [ ]:
# YOUR CODE HERE
def sigmoid(x):
    return np.maximum(np.minimum(1 / (1 + np.exp(-x)), 0.9999), 0.0001)


def cost_function(x, y, theta):
    t = x.dot(theta)
    # return —1 * np.sum(y * np.log(sigmoid(t)) + (1 — y) * np.log(1 — sigmoid(t))) / x.shape[0]


### Implement the Logistic regression using sklearn (2 points)

As there is imbalance in the class distribution, add weightage to the Logistic regression.

* Find the accuracy with class weightage in Logistic regression
* Find the accuracy without class weightage in Logistic regression

Hint: [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [ ]:
def calculate_metrics(y_true, y_pred):
    """
    Calculate classification metrics using pandas Series operations
    """
    # Convert inputs to pandas Series if they aren't already
    y_true = pd.Series(y_true)
    y_pred = pd.Series(y_pred)

    # Calculate metrics using pandas operations
    TP = sum((y_true == 1) & (y_pred == 1))
    TN = sum((y_true == 0) & (y_pred == 0))
    FP = sum((y_true == 0) & (y_pred == 1))
    FN = sum((y_true == 1) & (y_pred == 0))

    # Calculate derived metrics
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = (
        2 * (precision * recall) / (precision + recall)
        if (precision + recall) > 0
        else 0
    )

    return pd.Series(
        {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1_score,
            "TP": TP,
            "TN": TN,
            "FP": FP,
            "FN": FN,
        }
    )


In [ ]:
# With weightage
# YOUR CODE HERE
classifier_no_weights = LogisticRegression(random_state=101)
classifier_no_weights.fit(X_train, y_train)
y_pred_no_weights = classifier_no_weights.predict(X_test)

# calculate the accuracy too.


In [ ]:
# Without weightage
# YOUR CODE HERE

# Calculate class weights
class_weights = dict(
    df['SeriousDlqin2yrs'].value_counts(normalize=True).apply(lambda x: df.shape[0]/(2*df.shape[0]*x))
)


In [ ]:
classifier_with_weights = LogisticRegression(random_state=101, class_weight=class_weights)
classifier_with_weights.fit(X_train, y_train)
y_pred_with_weights = classifier_with_weights.predict(X_test)


In [ ]:
# Create imbalanced dataset using pandas
np.random.seed(42)
n_samples = 1000
n_class_0 = int(0.93 * n_samples)
n_class_1 = n_samples - n_class_0

# Create DataFrame with features and target
df = pd.DataFrame(
    {
        "feature1": np.concatenate(
            [np.random.normal(0, 1, n_class_0), np.random.normal(2, 1, n_class_1)]
        ),
        "feature2": np.concatenate(
            [np.random.normal(0, 1, n_class_0), np.random.normal(2, 1, n_class_1)]
        ),
        "target": np.concatenate([np.zeros(n_class_0), np.ones(n_class_1)]),
    }
)

# Split features and target
X = df[["feature1", "feature2"]]
y = df["target"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model without weights
model_no_weights = LogisticRegression(random_state=42)
model_no_weights.fit(X_train, y_train)
y_pred_no_weights = model_no_weights.predict(X_test)

# Calculate class weights using pandas
class_weights = dict(
    df["target"]
    .value_counts(normalize=True)
    .apply(lambda x: n_samples / (2 * n_samples * x))
)

# Train model with weights
model_with_weights = LogisticRegression(random_state=42, class_weight=class_weights)
model_with_weights.fit(X_train, y_train)
y_pred_with_weights = model_with_weights.predict(X_test)

# Calculate metrics for both models
metrics_no_weights = calculate_metrics(y_test, y_pred_no_weights)
metrics_with_weights = calculate_metrics(y_test, y_pred_with_weights)

# Create DataFrame with all metrics
metrics_df = pd.DataFrame(
    {"Without Weights": metrics_no_weights, "With Weights": metrics_with_weights}
)

# Create confusion matrices using pandas crosstab
conf_matrix_no_weights = pd.crosstab(
    y_test, pd.Series(y_pred_no_weights), rownames=["Actual"], colnames=["Predicted"]
)

conf_matrix_with_weights = pd.crosstab(
    y_test, pd.Series(y_pred_with_weights), rownames=["Actual"], colnames=["Predicted"]
)

# Print results
print("Class Distribution in Training Data:")
print(df["target"].value_counts(normalize=True).mul(100).round(2), "\n")

print("Class Weights Used:")
print(pd.Series(class_weights), "\n")

print("Model Performance Metrics:")
print(metrics_df.round(3), "\n")

print("\nConfusion Matrix - Without Weights:")
print(conf_matrix_no_weights)

print("\nConfusion Matrix - With Weights:")
print(conf_matrix_with_weights)

# Optional: Create visualizations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot feature distribution
sns.scatterplot(data=df, x="feature1", y="feature2", hue="target", ax=ax1, alpha=0.6)
ax1.set_title("Feature Distribution")

# Plot metrics comparison
metrics_df.loc[["accuracy", "precision", "recall", "f1_score"]].plot(kind="bar", ax=ax2)
ax2.set_title("Model Metrics Comparison")
ax2.set_ylabel("Score")
plt.tight_layout()


### Credit scoring (1 point)

When scaling the model into a scorecard, we will need both the Logistic Regression coefficients from model fitting as well as the transformed WoE values. We will also need to convert the score from the model from the log-odds unit to a points system.
For each independent variable Xi, its corresponding score is:

$Score = \sum_{i=1}^{n} (-(β_i × WoE_i + \frac{α}{n}) × Factor + \frac{Offset}{n})$

Where:

βi — logistic regression coefficient for the variable Xi

α — logistic regression intercept

WoE — Weight of Evidence value for variable Xi

n — number of independent variable Xi in the model

Factor, Offset — known as scaling parameter

  - Factor = pdo / ln(2); pdo is points to double the odds
  - Offset = Round_of_Score - {Factor * ln(Odds)}

In [ ]:
# Scaling factors
factor = 20 / np.log(2)
offset = 600 - (factor * np.log(50))
factor, offset


In [ ]:
# YOUR CODE HERE


### Performance Metrics (1 point)

#### Precision

In [ ]:
# YOUR CODE HERE


#### Recall

In [ ]:
# YOUR CODE HERE


#### Classification Report

In [ ]:
# YOUR CODE HERE


#### Confusion matrix

In [ ]:
# YOUR CODE HERE


### Report Analysis

* Comment on the performance of the model with weightage and without weightage
* Have you tried implementing Logistic regression with normal features instead of WOE ?
  - Compare the classification report for both implementations

## SHAP implementation for Logistic Regression (Optional)

Implement SHAP and understand the underlying reasons or factors that are maximally influencing the model's predictions.

**SHAP (SHapley Additive exPlanations)** is a game theoretic approach to explain the output of any machine learning model. It connects optimal credit allocation with local explanations using the classic Shapley values from game theory and their related extensions.

In the below figure, we can see how the different feature values (Age, Sex, BMI, etc) are affecting the base value (0.1) to give the final output prediction (0.4). The base value or the expected value is the average of the model output over the training data X_train.

![](https://cdn.iisc.talentsprint.com/CDS/Images/Shap_model.png)

To understand how to compute and interpet Shapley-based explanations of a machine learning model, we will use the following plots:

- Force plot
- Feature importance plot
- Summary plot
- Dependence plot
- Clustering Shapley values

In [ ]:
!pip -qq install shap


In [ ]:
import shap
import warnings

warnings.filterwarnings("ignore")


### Force plot for individual instances (Optional)

We can visualize feature attributions such as Shapley values as “forces”. Each feature value is a force that either increases or decreases the prediction. The prediction starts from the baseline. The baseline for Shapley values is the average of all predictions.

In the plot, each Shapley value should be represented as an arrow that pushes to increase (positive value) or decrease (negative value) the prediction. These forces balance each other out at the actual prediction of the data instance.

**Hint:** Use `shap.force_plot()` function that takes three values:

- the base value (explainer.expected_value),
- the SHAP values, and
- the matrix of feature values

Show the SHAP force plots for two instances from the GiveMeSomeCredit dataset:

In [ ]:
# Instead of using the whole training set to estimate expected values, we summarize with
# a set of weighted kmeans, each weighted by the number of points they represent.
# summarize the background as K samples. Use 'shap.kmeans()' and
# store it in a variable 'xtrain_summary'

# YOUR CODE HERE


In [ ]:
# Convert the selected columns into list
feature_names = selected_columns.tolist()


In [ ]:
# Force plot for an instance
shap.initjs()
explainer = shap.KernelExplainer(log_reg.predict, xtrain_summary)
shap_value = explainer.shap_values(xtest.iloc[0, :].values)
shap.force_plot(
    explainer.expected_value,
    shap_value,
    xtest.iloc[0, :].values,
    feature_names=feature_names,
)


In the above plot:

- The output value **f(x)** is the prediction for that observation (the predicted output value of the first row in X_test is ≈ **0**, indicating low credit risk).
- The **base value**: is “the value that would be predicted if we did not know any features for the current output.” Here it is **0.004808**.
- Red/blue: Features that push the prediction value higher (to the right) are shown in red, and those pushing the prediction value lower are in blue.

The features are having risk decreasing effects.

In [ ]:
# Force plot for another instance

# YOUR CODE HERE


From the above plot, put your remarks on the output value **f(x)** i.e, the predicted output value for that observation is ≈ **0**.

Put your remarks on predicted risk.

Shapley values can be combined into global explanations. If we run SHAP for every instance, we get a matrix of Shapley values. This matrix has one row per data instance and one column per feature. We can interpret the entire model by analyzing the Shapley values in this matrix.

### SHAP Feature Importance (Optional)

The idea behind SHAP feature importance is simple: Features with large **absolute** Shapley values are important. Since we want the global importance, we average the absolute Shapley values per feature across the data. Next, we sort the features by decreasing importance and plot them.

Plot the SHAP feature importance for the logistic regression model trained before for predicting credit risk.

**Hint:** Use the `shap.summary_plot` function with `plot_type=”bar”` to produce the feature importance plot. It lists the most significant features in descending order. The top variables contribute more to the model than the bottom ones and thus have high predictive power.

In [ ]:
# Get SHAP values
shap_values = explainer.shap_values(xtest)


In [ ]:
# Plot the Feature importance. Use 'shap.summary_plot()'

# YOUR CODE HERE


For a more informative plot, we will look at the summary plot.

### SHAP Summary Plot (Optional)

The summary plot combines feature importance with feature effects:

* Each point on the summary plot shoukd indicate a Shapley value for a feature and an instance.
* The position on the y-axis should be determined by the feature and on the x-axis by the Shapley value.
* Overlapping points should be appearing jittered in y-axis direction, to get a sense of the distribution of the Shapley values per feature.

In [ ]:
# Show the Summary plot. Use 'shap.summary_plot()'

# YOUR CODE HERE


In the summary plot, we see first indications of the relationship between the value of a feature and the impact on the prediction. But to see the exact form of the relationship, we have to look at SHAP dependence plots.

### SHAP Dependence Plot (Optional)

The partial dependence plot shows the marginal effect of one or two features have on the predicted outcome of a machine learning model. It tells whether the relationship between the target and a feature is linear, monotonic or more complex.

To implement SHAP feature dependence plot:

* Pick a feature
* For each data instance, plot a point with the feature value on the x-axis and the corresponding Shapley value on the y-axis

**Hint:** In order to create a dependence plot, we use `shap.dependence_plot()` function. The function automatically includes another variable that the chosen variable interacts most with. The following plot should show the relationship between “age” and the target variable, and “age” interacts with “NumberOfTime60-89DaysPastDueNotWorse” feature frequently.

In [ ]:
# Show the Dependence plot. Use 'shap.dependence_plot()'

# YOUR CODE HERE


### Clustering Shapley Values (Optional)

We can cluster the data with the help of Shapley values. The goal of clustering is to find groups of similar instances.

SHAP clustering works by clustering the Shapley values of each instance. This means that we cluster instances by explanation similarity. All SHAP values have the same unit – the unit of the prediction space. We can use any clustering method. In the following code cell, you need to use hierarchical agglomerative clustering to order the instances.

The plot should consist of many force plots, each of which will explain the prediction of an instance. Rotate the force plots vertically and place them side by side according to their clustering similarity.

In [ ]:
# Show the Force plot for first 100 instances of xtest
shap.initjs()

# Use shap.force_plot()

# YOUR CODE HERE


Put your remarks here regarding the above plot

Also, we can show the force_plot() for the entire set using the below code cell. Note that it may take more than 30 minutes to run the below cell.

In [ ]:
# Force plot for entire xtest
# shap.initjs()
# shap.force_plot(explainer.expected_value, shap_values, xtest.values, feature_names = feature_names)
